= 학습 목표

0. 준비물
    0.1 파이썬 에디터 (PyCharm, Anaconda)
1. 기초
    1.1 함수형 프로그래밍 (lambda, map, filter)
    1.2 시간 관련 함수
2. 업비트 라이브러리 사용법
    2.0 업비트 API Key/Secret 발급 방법 (https://m.post.naver.com/viewer/postView.nhn?volumeNo=16270764&memberNo=40921089)
    2.1 업비트 API 기본 사용법
    2.2 주문시 가격 자리수 맞추기
    2.3 Bid/Ask (구매/판매)
    2.4 Dump (Asks all markets)
3. 변동성 돌파 전략
    3.1 전체 플로우
        - 매수 조건 : 0시의 시가 + (k * range)을 넘어 가면 매수
        - k = 최근 20일간의 noise ratio 평균
        - range = 최근24시간의 고가 - 최근24시간의 저가
        - 배팅 비율 = 이평선 스코어 * 2% Rule / 5
    3.2 전체 KRW 마켓 발라내기
    3.3 성장하는 마켓 발라내기
    3.4 각 성장하는 마켓 중 돌파 가격 정하기 (얼마 이상이면 사겠다


주의 : 요즘 코인 시장이 좋지 않아서 그대로 돌리면 손실 가능 유의. 대신, 이 전략을 주식 등에 적용 가능하니 연구해보면 좋을 것으로 보입니다.

In [ ]:
# 기초1 - map/lambda

s = [1, 2, 3, 4]
print(list(map(lambda x: x*x, s)))  # 결과를 맞춰보세요

In [ ]:
# 기초2 - map/lambda

d = [{'market': 'KRW-BTC'}, {'market': 'KRW-ONT'}]
print(list(map(lambda x: x['market'], d))) # 결과를 맞춰보세요

In [ ]:
# 기초3 - filter 사용법

d = [{'market': 'KRW-BTC'}, {'market': 'BTC-ONT'}]
m = map(lambda x: x['market'], d)
print(list(filter(lambda x: x.startswith('KRW'), m))) # 결과를 맞춰보세요

In [ ]:
# 현재 시간 알아내기

from pytz import timezone
from datetime import datetime

t = datetime.now(timezone('Asia/Seoul'))
print(t.hour)
print(t.minute)

In [ ]:
# 업비트 API 연동 방법

from upbitlib.upbit import Upbit
import json
from pprint import pprint

# API 연동은.. https://m.post.naver.com/viewer/postView.nhn?volumeNo=16270764&memberNo=40921089
upbit = Upbit('APIKey', 'Secret')

In [ ]:
# 업비트 API 사용 예제 -

pprint(json.dumps(upbit.get_markets(), indent=2))  # 코인 목록
pprint(json.dumps(upbit.get_candles_daily('KRW-BTC', '', 3), indent=2))  # BTC 가격
pprint(upbit.get_accounts())  # 내 계좌 조회

In [ ]:
# 업비트 주문시 가격 절삭 하기

def fix_price(price):
    _unit = {
        1: 0.01,
        10**1: 0.1,  # 10원 -> 12.123원 -> 12.1원
        10**2: 1,    # 100원 -> 101.34원 -> 101원
        10**3: 5,
        10**4: 10,
        5*10**4: 50,
        10**5: 100,
        10**6: 500,   # 700_0000원 ->
        2*10**6: 1000
    }
    for p in _unit:
        if price > p:
            price = (price // _unit[p]) * _unit[p]
    return price

pprint(fix_price(312000.12345))
pprint(fix_price(31312.15))
pprint(fix_price(31.1234))

In [ ]:
# 이제 레퍼리움 코인을 530원치 사볼까요?
market = 'KRW-RFR'
budget = 530
ticker = upbit.get_ticker(market) # 레퍼리움의 현재 가격 읽어오기
# pprint(ticker)
last_price = ticker[0]['trade_price'] * 1.001 # +0.1%

result = upbit.place_order(market, 'bid', fix_price(budget/last_price), last_price)
pprint(result)

In [ ]:
# 가지고 있는 코인 전부 팔아볼까요?

def sell(market, amount):
    print("sell market = {0} with amount = {1}".format(market, amount))
    # 비어 있는 함수

def dump_all():
    accounts_list = upbit.get_accounts()
    accounts_list = filter(lambda z: z['currency'] != 'KRW', accounts_list)
    for wallet in accounts_list:
        sell('KRW-{0}'.format(wallet['currency']), wallet['balance'])

dump_all()

변동성 돌파 전략

 - 매수 조건 : 0시의 시가 + (k * range)을 넘어 가면 매수
 - k = 최근 20일간의 noise ratio 평균
 - range = 최근24시간의 고가 - 최근24시간의 저가
 - 배팅 비율 = 이평선 스코어 * 2% Rule / 5

참고 그림 : https://m.post.naver.com/viewer/postView.nhn?volumeNo=16270764&memberNo=40921089

In [ ]:
# KRW 코인 목록 읽어오기

def candidate_coins():
    candidate_coin = map(lambda x: x['market'], upbit.get_markets())
    return filter(lambda x: x.startswith('KRW'), candidate_coin)

#print("list of candidate coins")
pprint(list(candidate_coins()))

In [ ]:
# 가격이 오르고 있는 코인만 사기 위해서, 상승 여부를 판단하는 로직

def is_growing_market(market):
    prices = upbit.get_candles_daily(market, '', 5)
    pprint(json.dumps(prices, indent=2))
    return prices[0]['trade_price'] > prices[-1]['trade_price']

pprint(is_growing_market('KRW-RFR'))

In [ ]:
# 각 코인을 얼마나 투자 해야 하는가?

def get_betting_ratio(market):
    '''
    3일~20일의 18개의 이동 평균선을 계산
    이동평균선 스코어 = 각 이동평균선을 넘은 개수/18
    e.g., 3일의 이동 평균선 = (1일전 종가 + 2일전 종가 + 3일전 종가)/3
          => 만약 현재 가격이 3일의 이동 평균 가격 보다 높으면 score 1/18 더한다
    '''
    prices = upbit.get_candles_daily(market, '', 21)
    score = 0
    if len(prices) < 21:
        return 0  # 신생 코인은 제외

    for period in range(3, 20):
        sum_prices = 0
        for j in range(0, period):
            sum_prices += prices[j+1]['trade_price']

        if sum_prices/period < prices[0]['opening_price']:
            score += 1/18.0
    return score

pprint(json.dumps(get_betting_ratio('KRW-BTC'), indent=2))
pprint(json.dumps(get_betting_ratio('KRW-RFR'), indent=2))  # 레퍼리움은 불안정 하기 때문에 원래 투자 금액의 1/10
pprint(json.dumps(get_betting_ratio('KRW-MFT'), indent=2))  # 메인프레임 코인은 상장한지 20일이 안되서 투자 안함

In [ ]:
# K값은 어떻게 구하나?
# range = (종가 - 시작가)
# 시작가 + range*k 이상이 되면 산다
#  - k = 최근 20일간의 noise ratio 평균

def get_k(market):
    prices = upbit.get_candles_daily(market, '', 20)[1:]
    price_noise = list(map(lambda p: 1 - abs(p['trade_price'] - p['opening_price']) / (p['high_price'] - p['low_price']), prices))
    return sum(price_noise) / len(price_noise)

print(get_k('KRW-MFT'))
print(get_k('KRW-BTC'))

In [ ]:
# 투자할 코인만 골라내기

trade_markets = list(candidate_coins())
coin_noise = {}
coin_betting_ratio = {}

# 오래 걸림 주의
for market in trade_markets:
    coin_noise[market] = get_k(market)
    coin_betting_ratio[market] = get_betting_ratio(market)

trade_markets = list(filter(lambda m: coin_betting_ratio[m] > 0, trade_markets))
pprint(trade_markets)

In [ ]:
# 이제 실제로 돌리기
already_buy = {}
BETTING_BUDGET = 5000

while True:
    for market in trade_markets:
        if market in already_buy:
            continue

        candles = upbit.get_candles_daily(market, '', 2)  # Today, Yesterday
        _range = candles[1]['high_price'] - candles[1]['low_price']

        today_opening = candles[0]['opening_price']
        today_current = candles[0]['trade_price']

        threshold = _range * coin_noise[market]

        over_ratio = today_current / (today_opening + threshold)
        if over_ratio > 1.0:
            print(market, "buy now!!! = ", over_ratio, " budget = ", BETTING_BUDGET * coin_betting_ratio[market])
            #buy(market, BETTING_BUDGET * coin_betting_ratio[market])
            already_buy[market] = True
            # 만약 현재 시가 기준으로 전날 등락폭 대비해서 올랐으면 사자

        t = datetime.now(timezone('Asia/Seoul'))
        if t.hour == 23 and t.minute > 45:  # 다음날 되기 전에 팔자
            dump_all()
            #exit(0)
    break # 예제를 돌리면 무한으로 돌기 때문에 한번만 실행

# crontab -> 0시에 bot.py 를 실행하라